In [202]:
from collections import namedtuple
from enum import Enum

In [203]:
Condition = Enum("Condition", ("CURE", "HEALTHY", "SICK", "DYING", "DEAD"))
Agent = namedtuple("Agent", ("name", "category"))

### Helper Functions

In [204]:
def pair_up(agent_listing: tuple) -> list: 
    """Divide the list of agents to pairs. 
     
    If there's an uneven number of agents, the last agent will remain the same.

    Parameters
    ----------
    agent_listing : tuple of Agent
        A listing (tuple in this case) in which each element is of the Agent
        type, containing a 'name' field and a 'category' field, with 'category' being
        of the type Condition.

    Returns
    -------
    updated_listing : list
        A list of couples of Agents - defining the meeting couple
       
          
    """
    paired_list = []
    for i in range (0, len(agent_listing), 2): 
        if i+1 < len(agent_listing):
            paired_list.append((agent_listing[i] , agent_listing[i+1]))
        else: 
            paired_list.append((agent_listing[i],))
    
    return paired_list


def get_better(agent):
    """decrease the condition of an agent by 1

    Parameters
    ----------
    agent : Agent - a single object with condition 

    Returns
    -------
    The agent with a better condition
    
    """
    new_val = agent.category.value - 1  
    return Agent(agent.name, Condition(new_val))



def get_worse(agent):
    """increase the condition of an agent by 1

    Parameters
    ----------
    agent : Agent - a single object with condition 

    Returns
    -------
    The agent with a worse condition
    
    """
    new_val = agent.category.value + 1  
    return Agent(agent.name, Condition(new_val))




def post_meetup_condition(pair: tuple) -> tuple:
    """Model the outcome of the meetings of pairs of agents.

    The pairs of agents are ((a[0], a[1]), (a[2], a[3]), ...). If there's an uneven
    number of agents, the last agent will remain the same.

    Notes
    -----
    assumes the tuple is a pair (not a single agent)

    Parameters
    ----------
    pair : tuple of two agents and their condition

    Returns
    -------
    updated_listing : list
        A list of Agents with their 'category' field changed according to the result
        of the meeting.
    """
    # if single
    if len(pair) == 1:
            return pair
    
    # both are CURE → no change
    if all(agent.category == Condition.CURE for agent in pair):
        return pair  

    # one is CURE, help the other
    if any(agent.category == Condition.CURE for agent in pair):
        # only decrease the non-CURE agents
        updated = tuple(
            agent if agent.category == Condition.CURE else get_better(agent)
            for agent in pair
        )
        return updated

    # else: no one is CURE → increase both
    updated = tuple(
        get_worse(agent) for agent in pair
    )
    return updated



### main function

In [205]:
def meetup(agent_listing: tuple) -> list:
    """Model the outcome of the meetings of pairs of agents.

    The pairs of agents are ((a[0], a[1]), (a[2], a[3]), ...). If there's an uneven
    number of agents, the last agent will remain the same.

    Notes
    -----
    The rules governing the meetings were described in the question. The outgoing
    listing may change its internal ordering relative to the incoming one.

    Parameters
    ----------
    agent_listing : tuple of Agent
        A listing (tuple in this case) in which each element is of the Agent
        type, containing a 'name' field and a 'category' field, with 'category' being
        of the type Condition.

    Returns
    -------
    updated_listing : list
        A list of Agents with their 'category' field changed according to the result
        of the meeting.
    """

    update_listing = [] 
    sick_and_cure = [] 

    for agent in agent_listing: 
        if agent.category == Condition.HEALTHY or agent.category == Condition.DEAD:
            update_listing.append(agent)
        else: 
            sick_and_cure.append(agent)

    # pairs for meetup
    paired_list = pair_up(sick_and_cure)

    #calculate the condition post meeting 
    for pair in paired_list:
        update_listing.append(post_meetup_condition(pair))
    
    return update_listing


# test_q2

In [206]:
""" Tests for question 2 - Spreading Virus """
from hw2_q2 import Agent, Condition, meetup

data0 = (
    Agent("Adam", Condition.SICK),
    Agent("Cure0", Condition.CURE),
    Agent("Cure1", Condition.CURE),
    Agent("Bob", Condition.HEALTHY),
    Agent("Alice", Condition.DEAD),
    Agent("Charlie", Condition.DYING),
    Agent("Vaccine", Condition.SICK),
    Agent("Darlene", Condition.DYING),
    Agent("Emma", Condition.SICK),
    Agent("Cure2", Condition.CURE),
)

data1 = (Agent("Buddy", Condition.CURE), Agent("Holly", Condition.DEAD))

data2 = (
    Agent("Zelda0", Condition.SICK),
    Agent("Zelda1", Condition.SICK),
    Agent("Zelda2", Condition.SICK),
    Agent("Zelda3", Condition.SICK),
    Agent("Zelda4", Condition.DEAD),
    Agent("Zelda5", Condition.HEALTHY),
)

data3 = (
    Agent("Mark", Condition.SICK),
    Agent("Mork", Condition.HEALTHY),
    Agent("Harry", Condition.DYING),
    Agent("Cure", Condition.CURE),
    Agent("Lora", Condition.SICK),
    Agent("Monica", Condition.SICK),
)

data4 = (Agent("Robert", Condition.SICK),)

data5 = ()


def test_data0():
    code_result = set(meetup(data0))
    true_result = {
        Agent(name="Adam", category=Condition.HEALTHY),
        Agent(name="Alice", category=Condition.DEAD),
        Agent(name="Bob", category=Condition.HEALTHY),
        Agent(name="Charlie", category=Condition.SICK),
        Agent(name="Cure0", category=Condition.CURE),
        Agent(name="Cure1", category=Condition.CURE),
        Agent(name="Cure2", category=Condition.CURE),
        Agent(name="Darlene", category=Condition.DEAD),
        Agent(name="Emma", category=Condition.HEALTHY),
        Agent(name="Vaccine", category=Condition.DYING),
    }
    assert code_result == true_result


def test_data1():
    code_result = set(meetup(data1))
    true_result = set(data1)
    assert code_result == true_result


def test_data2():
    code_result = set(meetup(data2))
    true_result = {
        Agent("Zelda0", Condition.DYING),
        Agent("Zelda1", Condition.DYING),
        Agent("Zelda2", Condition.DYING),
        Agent("Zelda3", Condition.DYING),
        Agent("Zelda4", Condition.DEAD),
        Agent("Zelda5", Condition.HEALTHY),
    }
    assert code_result == true_result


def test_data3():
    code_result = set(meetup(data3))
    true_result = {
        Agent("Mark", Condition.DYING),
        Agent("Mork", Condition.HEALTHY),
        Agent("Harry", Condition.DEAD),
        Agent("Cure", Condition.CURE),
        Agent("Lora", Condition.HEALTHY),
        Agent("Monica", Condition.SICK),
    }
    assert code_result == true_result


def test_data4():
    code_result = set(meetup(data4))
    true_result = {Agent("Robert", Condition.SICK)}
    assert code_result == true_result


def test_data5():
    code_result = meetup(data5)
    true_result = []
    assert code_result == true_result


if __name__ == "__main__":
    methods = [f"test_data{num}" for num in range(6)]
    errors = []

    for method in methods:
        try:
            eval(method)
        except AssertionError as e:
            errors.append(f"Failed when testing method 'test_{method}': {e}")

    if errors:
        raise AssertionError(errors)
    else:
        print("Tests pass successfully.")


Tests pass successfully.
